[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/vector-search-lab/)

# Step 1: Setup prerequisites

Replace `<MONGODB_URI>` with your **MongoDB connection string**

In [1]:
import os
from pymongo import MongoClient

In [2]:
# Retain the quotes ("") when pasting the URI
MONGODB_URI = os.environ.get("MONGODB_URI")
# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI, appname="devrel.workshop.vector_search")
# Check the connection to the server
mongodb_client.admin.command("ping")

{'ok': 1}

### **Do not change the values assigned to the variables below**

In [3]:
# Database name
DB_NAME = "mongodb_genai_devday"
# Collection name
COLLECTION_NAME = "books"
# Name of the vector search index
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

📚 https://pymongo.readthedocs.io/en/stable/tutorial.html#getting-a-database

In [4]:
# Connect to the `DB_NAME` database.
# Use the `mongodb_client` instantiated above.
db = mongodb_client[DB_NAME]

📚 https://pymongo.readthedocs.io/en/stable/tutorial.html#getting-a-collection

In [5]:
# Connect to the `COLLECTION_NAME` collection.
# Use the `mongodb_client`, database and collection variables defined above.
collection = db[COLLECTION_NAME]

In [6]:
SERVERLESS_URL = os.environ.get("SERVERLESS_URL")

# Step 2: Import data into MongoDB

In [7]:
import requests

In [8]:
response = requests.post(
    url=SERVERLESS_URL, json={"task": "import_vs_data", "data": MONGODB_URI}
)
if response.status_code == 200:
    print(
        f"{collection.count_documents({})} documents ingested into the books collection."
    )
else:
    raise Exception(response.json())

50 documents ingested into the books collection.


# Step 3: Generating embeddings

In [9]:
# You may see a warning upon running this cell. You can ignore it.
from sentence_transformers import SentenceTransformer
from PIL import Image

/home/vernica/proj/mongodb/genai-devday-notebooks/vector-search-lab/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Load a multimodal embedding model using the Sentence Transformers library
embedding_model = SentenceTransformer("clip-ViT-B-32")

### For images

📚 https://huggingface.co/sentence-transformers/clip-ViT-B-32#usage

In [11]:
image_url = "https://images.isbndb.com/covers/77/44/9780789467744.jpg"
# Load the image from the URL above
image = Image.open(requests.get(image_url, stream=True).raw)
# Embed the `image` using the `embedding_model` instantiated above and return the embedding as a list
# An array can be converted to a list using the `tolist()` method
embedding = embedding_model.encode(image).tolist()

In [12]:
print(embedding)

[0.2504979968070984, 0.19034789502620697, 0.3679232895374298, -0.11894109100103378, -0.2693753242492676, -0.3255960941314697, 0.41852352023124695, -0.09661254286766052, 0.20106720924377441, 0.28391698002815247, 0.5291520953178406, 0.06442062556743622, -0.21868611872196198, 0.4178038239479065, 0.3735407292842865, -0.265012264251709, 0.27570921182632446, 0.368049681186676, 0.2726353704929352, -0.3859325647354126, -0.8549752831459045, 0.18848520517349243, -0.023599576205015182, 0.8512741923332214, -0.38916340470314026, -0.056455984711647034, 0.234907329082489, 0.3864738643169403, -0.23601725697517395, 0.14198987185955048, -0.31521064043045044, 0.12545916438102722, -0.28378358483314514, 0.013077053241431713, -0.09076753258705139, -0.061463769525289536, 0.17084595561027527, 8.89115035533905e-05, 0.0729936957359314, -0.3049669861793518, -0.10878414660692215, 0.17290841042995453, 0.061156220734119415, 0.23829959332942963, 0.08692534267902374, -1.163448452949524, -0.3901008069515228, -0.042774

### For text

In [13]:
text = "Puppy Preschool: Raising Your Puppy Right---Right from the Start!"
# Use the same `embedding_model` to embed a piece of text
embedding = embedding_model.encode(text).tolist()

In [14]:
print(embedding)

[-0.1250314712524414, -0.34212160110473633, -0.2654053866863251, 0.01220320537686348, -0.13794109225273132, -0.05129276216030121, 0.4078459143638611, 0.18069583177566528, -0.2614646255970001, -0.37048280239105225, -0.6959188580513, -0.17315219342708588, -0.15652313828468323, -0.2016838788986206, 0.055937446653842926, 0.002341795712709427, -0.5091979503631592, -0.07782120257616043, 0.018008744344115257, 0.20341597497463226, -0.484042763710022, -0.32365942001342773, 0.6865009665489197, 0.41892433166503906, 0.006406545639038086, -0.03999973461031914, 0.27117007970809937, 0.32391566038131714, -0.18733251094818115, -0.15335270762443542, 0.14605751633644104, -0.06396700441837311, 0.09700703620910645, -0.33823055028915405, -0.5776907205581665, 0.01687849499285221, -0.08471284061670303, 0.11571841686964035, -0.5809029340744019, -0.35574308037757874, 0.10551197826862335, 0.07549693435430527, -0.024137230589985847, 0.09987793117761612, 0.17345871031284332, -0.10375776886940002, 0.140001997351646

# Step 4: Adding embeddings to existing data in Atlas

In [15]:
from typing import List, Dict, Optional

In [16]:
# Field in the documents to embed-- in this case, the book cover
field_to_embed = "cover"
# Name of the embedding field to add to the documents
embedding_field = "embedding"

In [17]:
def get_embedding(content: str, mode: str) -> List[float]:
    """
    Generate embeddings

    Args:
        content (str): Content to embed
        mode (str): Content mode (Can be one of "image" or "text")

    Returns:
        List[float]: Embedding of the content as a list.
    """
    # If an image URL is provided, first load the image
    if mode == "image":
        content = Image.open(requests.get(content, stream=True).raw)
    return embedding_model.encode(content).tolist()

📚 https://www.mongodb.com/docs/manual/tutorial/query-documents/#select-all-documents-in-a-collection

In [18]:
# Query for all documents in the `collection` collection.
results = collection.find({})

📚 **$set:** https://www.mongodb.com/docs/manual/reference/operator/update/set/#syntax

📚 **update_one():** https://www.mongodb.com/docs/manual/reference/method/db.collection.updateOne/#definition

In [19]:
# Update each document in the `collection` collection with embeddings
for result in results:
    content = result[field_to_embed]
    # Use the `get_embedding` function defined above to embed the `content`
    # Note that `content` contains the cover image URL for the book 
    embedding = get_embedding(content, "image")

    
    # Filter for the document where the `_id` field is equal to the `_id` of the current document
    filter = {"_id": result["_id"]}
    # Set the `embedding_field` field to the value `embedding` using the `$set` operator
    update = {"$set": {embedding_field: embedding}}
    # Update the documents in the `collection` collection inplace using the `update_one()` operation
    # Get the right document `_id` using the `filter` and apply the `update`
    collection.update_one(filter, update)

# Step 5: Create a vector search index

In [20]:
# Create vector index definition specifying:
# path: Path to the embeddings field
# numDimensions: Number of embedding dimensions- depends on the embedding model used
# similarity: Similarity metric. One of cosine, euclidean, dotProduct.
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
            }
        ]
    },
}

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_search_index

In [21]:
# Create a vector search index with the above definition for the `collection` collection
collection.create_search_index(model)

'vector_index'

# Step 6: Perform vector search queries

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Filter Example")

In [22]:
# Define a function to retrieve relevant documents for a user query using vector search
def vector_search(
    user_query: str, mode: str, filter: Optional[Dict] = {}
) -> None:
    """
    Retrieve relevant documents for a user query using vector search.

    Args:
    user_query (str): The user's query (can be a piece of text or a link to an image)
    mode (str): Query mode (image or text)
    filter (Optional[Dict], optional): Optional vector search pre-filter
    """
    # Generate embedding for the `user_query` using the `get_embedding` function defined in Step 4
    query_embedding = get_embedding(user_query, "text")

    # Define an aggregation pipeline consisting of a $vectorSearch stage, followed by a $project stage
    # Set the number of candidates to 50 and only return the top 5 documents from the vector search
    # Set the `filter` field in the $vectorSearch stage to the value `filter` passed to the function
    # In the $project stage, exclude the `_id` field, include these fields: `title`, `cover`, `year`, `pages`, and the `vectorSearchScore`
    # NOTE: Use variables defined previously for the `index`, `queryVector` and `path` fields in the $vectorSearch stage
    pipeline = [
        { "$vectorSearch": {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "path": embedding_field,
            "queryVector": query_embedding,
            "filter": filter,
            "numCandidates": 50,
            "limit": 5
            },
        },
        { "$project": {
            "_id": 0,
            "title": 1,
            "cover": 1,
            "year": 1,
            "pages": 1,
            "score": { "$meta": "vectorSearchScore" }
            }
        }
    ]

    # Execute the aggregation `pipeline` and store the results in `results`
    results = collection.aggregate(pipeline)

    # Print book title, score, and cover image
    for book in results:
        cover = Image.open(requests.get(book.get("cover"), stream=True).raw).resize((100,150))
        print(f"{book.get('title')}({book.get('year')}, {book.get('pages')} pages): {book.get('score')}")
        display(cover)

In [23]:
# Test the vector search with a text query
vector_search("A peaceful and uplifting atmosphere", "text")

# Also try these text queries:
# - Healthy living and fitness
# - A man wearing a golden crown
# - Adorable fluffy creatures
# - Winning business agreement
# - A Winter's Tale on a Spanish Isle

In [24]:
# Test the vector search with an image query
vector_search("https://images.isbndb.com/covers/26/12/9780385412612.jpg", "image")

# Also try these image queries:
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/1.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/2.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/3.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/4.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/5.png

# Step 7: Adding pre-filters to your vector search

### Filter for books that were published after the year `2002`

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [25]:
# Modify the vector search index `model` from Step 5 to include the `year` field as a `filter` field
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
            },
            {
                "type": "filter",
                "path": "year"
            }
        ]
    },
}

In [26]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Check that the index update is complete before proceeding. To do so, ensure that the status of the index shows "Ready" in the Atlas UI.

📚 https://www.mongodb.com/docs/manual/reference/operator/query/gte/#syntax

In [27]:
# Create a filter definition to filter for books where the `year` field is greater than `2002` using the `$gte` operator
filter = {"year": {"$gte": 2002} }
# Pass the `filter` as an argument to the `vector_search` function.
# Notice how this filter is incorporated in the `pipeline` in the `vector_search` function.
vector_search("A peaceful and uplifting atmosphere", "text", filter)

### Filter for books that were published after the year `2002` and under `250` pages

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [28]:
# Modify the vector search index `model` from Step 5 to include `year` and `pages` as filter fields
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
            },
            {
                "type": "filter",
                "path": "year"
            },
            {
                "type": "filter",
                "path": "pages"
            }
        ]
    },
}

In [29]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Check that the index update is complete before proceeding. To do so, ensure that the status of the index shows "Ready" in the Atlas UI.

📚 https://www.mongodb.com/docs/manual/reference/operator/query/lte/#mongodb-query-op.-lte

In [30]:
# Create a filter definition to filter for books where the `year` field is greater than or equal to `2002` and the `pages` field is less than or equal to 250
# Use the `$gte` and `$lte` operators
filter = {"$and": [{"year": { "$gte": 2002}}, {"pages": {"$lte": 250}}]}
# Pass the `filter` as an argument to the `vector_search` function.
# Notice how this filter is incorporated in the `pipeline` in the `vector_search` function.
vector_search("A peaceful and uplifting atmosphere", "text", filter)

# Step 8: Changing the similarity metric

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#syntax

In [31]:
# Modify the vector search index `model` from Step 5 to change the similarity metric to `dotProduct`
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "dotProduct",
            }
        ]
    },
}

In [32]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Check that the index update is complete before proceeding. To do so, ensure that the status of the index shows "Ready" in the Atlas UI.

In [33]:
# Perform a vector search
# Note any differences in the results due to the different similarity metric
vector_search("A peaceful and uplifting atmosphere", "text")

# 🦹‍♀️ Enable vector quantization

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#syntax

In [34]:
# Modify the vector search index `model` from Step 5 to use `scalar` quantization
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
                "quantization": "scalar"
            }
        ]
    },
}

In [35]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

# 🦹‍♀️ Hybrid search

In [36]:
# Name of the full-text search index
ATLAS_FTS_INDEX_NAME = "fts_index"

In [37]:
# Create full-text search index definition specifying the field mappings
model = {
    "name": ATLAS_FTS_INDEX_NAME,
    "type": "search",
    "definition": {
        "mappings": {"dynamic": False, "fields": {"title": {"type": "string"}}}
    },
}

In [38]:
# Create a full-text search index with the above definition for the `collection` collection
collection.create_search_index(model=model)

'fts_index'

In [39]:
# Reset the vector search index to the original vector search index definition
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
            }
        ]
    },
}

# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Refer to our [documentation](https://www.mongodb.com/docs/atlas/atlas-vector-search/tutorials/reciprocal-rank-fusion/#about-the-query) for a detailed explanation of the hybrid search query below.

In [40]:
def hybrid_search(
    user_query: str, vector_weight: float, full_text_weight: float
) -> None:
    """
    Retrieve relevant documents for a user query using hybrid search.

    Args:
        user_query (str): User query string
        vector_weight (float): Weight of vector search in the final search results
        full_text_weight (float): Weight of full-text search in the final search results
    """
    pipeline = [
        # Vector search stage
        {
            "$vectorSearch": {
                "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
                "path": "embedding",
                "queryVector": get_embedding(user_query, "text"),
                "numCandidates": 50,
                "limit": 10,
            }
        },
        # Group all documents returned by the vector search into a single array named `docs`
        {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
        # Unwinds the array of documents in the `docs` field and stores the position of the document in the `docs` array, in a field named `rank`
        {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
        # Adds a new field named `vs_score` that contains the reciprocal rank score for each document in the `docs` array
        {
            "$addFields": {
                "vs_score": {
                    "$multiply": [
                        vector_weight,
                        {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                    ]
                }
            }
        },
        # Includes only the fields `_id`, `title`, and `cover` and the `vs_score` in the results
        {
            "$project": {
                "vs_score": 1,
                "_id": "$docs._id",
                "title": "$docs.title",
                "cover": "$docs.cover",
            }
        },
        # Union with a full-text search stage
        {
            "$unionWith": {
                "coll": COLLECTION_NAME,
                "pipeline": [
                    # Full-text search
                    {
                        "$search": {
                            "index": ATLAS_FTS_INDEX_NAME,
                            "text": {"query": user_query, "path": "title"},
                        }
                    },
                    # Limits the number of documents returned by the full-text search to 10
                    {"$limit": 10},
                    # Group all documents returned by the full-text search into a single array named `docs`
                    {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
                    # Unwinds the array of documents in the `docs` field and stores the position of the document in the `docs` array, in a field named `rank`
                    {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
                    # Adds a new field named `fts_score` that contains the reciprocal rank score for each document in the `docs` array
                    {
                        "$addFields": {
                            "fts_score": {
                                "$multiply": [
                                    full_text_weight,
                                    {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                                ]
                            }
                        }
                    },
                    # Includes only the fields `_id`, `title`, and `cover` and the `fts_score` in the results
                    {
                        "$project": {
                            "fts_score": 1,
                            "_id": "$docs._id",
                            "title": "$docs.title",
                            "cover": "$docs.cover",
                        }
                    },
                ],
            }
        },
        # The following stages combine the results from vector and full-text search into a single ranked list
        # Groups the documents in the results from the preceding stages by `title`, `vs_score`, and `fts_score`
        {
            "$group": {
                "_id": "$_id",
                "title": {"$first": "$title"},
                "vs_score": {"$max": "$vs_score"},
                "fts_score": {"$max": "$fts_score"},
                "cover": {"$first": "$cover"},
            }
        },
        # Includes only the fields `_id`, `title`, `cover`, `vs_score` and `fts_score` in the results
        {
            "$project": {
                "_id": 1,
                "title": 1,
                "vs_score": {"$ifNull": ["$vs_score", 0]},
                "fts_score": {"$ifNull": ["$fts_score", 0]},
                "cover": 1,
            }
        },
        # Adds a field named `score` that contains the sum of `vs_score` and `fts_score` to the results
        {
            "$project": {
                "score": {"$add": ["$fts_score", "$vs_score"]},
                "_id": 1,
                "title": 1,
                "vs_score": 1,
                "fts_score": 1,
                "cover": 1,
            }
        },
        # Sorts the results by descending order of `score`
        {"$sort": {"score": -1}},
        # Limits the output to 5 results only
        {"$limit": 5},
    ]

    results = collection.aggregate(pipeline)

    # Print book title, scores, and cover image
    for book in results:
        cover = Image.open(requests.get(book["cover"], stream=True).raw).resize(
            (100, 150)
        )
        print(
            f"{book.get('title')}, VS Score: {book.get('vs_score')}, FTS Score: {book.get('fts_score')}"
        )
        display(cover)

In [41]:
# Test the hybrid search query with a weight of 1.0 for vector search and 0.0 for full-text search
hybrid_search(
    user_query="A Winter's Tale on a Spanish Isle",
    vector_weight=1.0,
    full_text_weight=0.0,
)

In [42]:
# Test the hybrid search query with a weight of 0.5 for vector search and 0.5 for full-text search
hybrid_search(
    user_query="A Winter's Tale on a Spanish Isle",
    vector_weight=0.5,
    full_text_weight=0.5,
)